In [1]:
import jieba 
from nltk.parse.corenlp import CoreNLPParser
parser = CoreNLPParser('http://localhost:9001')
from nltk.tree import Tree
import numpy as np

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import re
import time
import pandas as pd
import os
from collections import Counter
import datetime
from opencc import OpenCC

In [3]:
import sys
sys.path.insert(0, "/Users/rikeion/Desktop/Anti_Money_Laundry/zh-NER-TF-master")

In [4]:
#接下來串接
import tensorflow as tf
import numpy as np
import os, argparse, time, random
from model import BiLSTM_CRF
from utils import str2bool, get_logger, get_entity
from data import read_corpus, read_dictionary, tag2label, random_embedding, vocab_build

# 新聞網站爬蟲 + 寫入檔案

In [5]:
def get_html(url):
    #模擬訪問頁面的函數
    try:
        user_agent = 'Mozilla/5.0'
        resp = requests.get(url, headers={'User-Agent': user_agent}, timeout = 30) #回傳為一個request.Response的物件
        resp.endcoding = 'utf8'
        #response = resp.content.decode()
        #c = resp.status_code
        return resp.text
    except:
        return 'ERROR'

In [34]:
def get_apple_news(keyword):
    #函數用來爬取特定關鍵字下的新聞內容，函數內可以再設定要爬取的頁數（或之後用時間來選），但蘋果日報需要先登入才能用（這部分還沒寫自動化）
    page_num = 10
    url_list = []
    for i in range(page_num):
        html = get_html('https://tw.appledaily.com/search/ajaxresult?querystrS='+ keyword +'&page='+str(i+1))
        json_parse = json.loads(html)
        for j in json_parse:
            url_list.append(j['sharing']['url'])
        #print(json_parse)
    #print(url_list)
    #開始訪問每個新聞連結並取得內容，最後寫進檔案
    w = open(keyword + "_all_apple_news.txt", 'w')
    for i in url_list:
        text = get_html(i)
        html = etree.HTML(text)
        news_content = html.xpath('//*[@id="article"]/div[1]/div/main/article/div[2]/div[2]/article/div/p[1]/text()')
        if news_content == []:
            news_content = html.xpath('//*[@id="article"]/div[1]/div/main/article/div[3]/article/div/p[1]/text()')#回傳list
        #把list變成一個字串
        news_content_str = str() 
        for i in news_content:
            news_content_str += i
        w.write(news_content_str)
        w.write('\n')
    w.close()
    
    return "Finish"
    

In [35]:
def get_china_times_news(keyword):
    #函數用來爬取特定關鍵字下的新聞內容，函數內可以再設定要爬取的頁數（或之後用時間來選）
    page_num = 10
    url_list = []
    #取得指定搜尋頁數裡的新聞url，並存成url_list
    for i in range(page_num):
        html = get_html('https://www.chinatimes.com/search/'+ keyword +'?page='+str(i+1)+'&chdtv')
        tree = etree.HTML(html)
        for j in range(13):
            url = tree.xpath('/html/body/div[2]/div/div[2]/div/section/div/ul/li['+ str(j+1) +']/div/div/div/h3/a/@href')
            url_list.append("".join(url))
    while '' in url_list: #廣告是空的，每一頁13個url中有3個是廣告，但抓下來都是空的，就去掉
        url_list.remove('')
    for i in range(len(url_list)):
        url_list[i] += '?chdtv'
    print(url_list)
     
    #開始訪問每個新聞連結並取得內容，最後寫進檔案
    w = open(keyword + "_all_china_times_news.txt", 'w')
    for i in url_list:
        text = get_html(i)
        html = etree.HTML(text)
        news_header = html.xpath('//*[@id="page-top"]/div/div[2]/div/div/article/div/header/h1/text()')
        news_content = html.xpath('//*[@id="page-top"]/div/div[2]/div/div/article/div/div[1]/div[2]/div[2]/div[2]//text()')
        news_content_str = str()
        news_content_str += "".join(news_header)
        news_content_str += "\n"
        for j in news_content:
            if j[0] != "\n":
                news_content_str += j
        w.write(news_content_str)
        #print(news_content_str)
        w.write('\n')
        #time.sleep(1)應該不是一次太多request的問題
    
    print(news_content_str)
    w.close()
    return 'yeah'




In [36]:
def get_link(keyword,idx=2):
    keyword_link = []
    url = "https://www.ettoday.net/news_search/doSearch.php?keywords="
    for i in keyword:
        keyword_link.append(url+i+'&idx='+str(idx))
    return keyword_link

In [37]:
def get_page(string):
    page = str()
    flag=1
    for i in range(len(string)):
        if string[i]=='頁':
            flag=1
        if flag==0:
            page+=(string[i])
        if string[i]=='共':
            flag = 0
    return int(page)

In [38]:
def get_news_link(url):
    try:
        news_link = []
        total_page = 10000
        page_num = 1
        while page_num <= total_page:
            html = get_html(url)
            soup = BeautifulSoup(html,"html.parser")
            sel = soup.select("div.result_archive div.box_1 a")
            pages = soup.select("div.menu_page p")
            total_page = get_page(pages[0].text)
            if not total_page:
                return '您所輸入的查詢條件查無資料，請重新設定查詢條件！'
            url = url+"&page="+str(page_num)
            page_num+=1
            for i in sel:
                news_link.append(i["href"])
        return news_link
    except:
        return '您所輸入的查詢條件查無資料，請重新設定查詢條件！'

In [39]:
def get_news_articles(url):
    html = get_html(url)
    soup = BeautifulSoup(html,"html.parser")
    sel = soup.select("div.story p")
    news_articles = str()
    for i in sel:
        if i.text!='' and ('strong' not in str(i)):
            news_articles+=i.text
    return news_articles

In [40]:
def get_all_news(news_link):
    news = []
    if type(news_link)==str:
        return '您所輸入的查詢條件查無資料，請重新設定查詢條件！'
    for i in news_link:
        news.append(get_news_articles(i))
    return news

In [ ]:
def create_news_dict(keyword):
    keyword_link = get_link(keyword)
    news_dict = {}
    for i,link in enumerate(keyword_link):
        news_link = get_news_link(link)
        news = get_all_news(news_link)
        news_dict[keyword[i]] = news
    return news_dict

# 自檔案讀取文章

In [5]:
def read_text():
    #函數讀取爬蟲下來的新聞
    r = open('test_news.txt', 'r')
    text = ''
    for i in r:
        text += i
    sentence = ''
    article = []
    for i in text:   
        if (i != '。') and (i != '\n'):
            sentence += i
        else:
            if sentence != '':
                article.append(sentence)
            sentence = ''
    return article

# 使用NER自文章中取出名字

In [5]:
## Session configuration
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # default: 0
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.2  # need ~700MB GPU memory

parser = argparse.ArgumentParser(description='BiLSTM-CRF for Chinese NER task')
#parser.add_argument('--update_train_data', type=str, default='data_path', help='update training data set')

parser.add_argument('--train_data', type=str, default='data_path', help='train data source')
parser.add_argument('--test_data', type=str, default='data_path', help='test data source')
parser.add_argument('--batch_size', type=int, default=64, help='#sample of each minibatch')
parser.add_argument('--epoch', type=int, default=40, help='#epoch of training')#default=40

parser.add_argument('--hidden_dim', type=int, default=300, help='#dim of hidden state')
parser.add_argument('--optimizer', type=str, default='Adam', help='Adam/Adadelta/Adagrad/RMSProp/Momentum/SGD')
parser.add_argument('--CRF', type=str2bool, default=True, help='use CRF at the top layer. if False, use Softmax')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--clip', type=float, default=5.0, help='gradient clipping')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout keep_prob')
parser.add_argument('--update_embedding', type=str2bool, default=True, help='update embedding during training')
parser.add_argument('--pretrain_embedding', type=str, default='random', help='use pretrained char embedding or init it randomly')
parser.add_argument('--embedding_dim', type=int, default=300, help='random init char embedding_dim')
parser.add_argument('--shuffle', type=str2bool, default=True, help='shuffle training data before each epoch')
parser.add_argument('--mode', type=str, default='demo', help='train/test/demo/update')
parser.add_argument('--demo_model', type=str, default='1559725764', help='model for test and demo')
args = parser.parse_args(args=[])

In [6]:
%cd zh-NER-TF-master

/Users/rikeion/Desktop/Anti_Money_Laundry/zh-NER-TF-master


In [7]:
## get char embeddings
word2id = read_dictionary(os.path.join('.', args.train_data, 'word2id.pkl'))
if args.pretrain_embedding == 'random':
    embeddings = random_embedding(word2id, args.embedding_dim)
else:
    embedding_path = 'pretrain_embedding.npy'
    embeddings = np.array(np.load(embedding_path), dtype='float32')


train_path = os.path.join('.', args.train_data, 'train_data')
test_path = os.path.join('.', args.test_data, 'test_data')
train_data = read_corpus(train_path)
test_data = read_corpus(test_path); test_size = len(test_data)

paths = {}
timestamp = str(int(time.time())) if args.mode == 'train' else args.demo_model
output_path = os.path.join('.', args.train_data+"_save", timestamp)

if not os.path.exists(output_path): os.makedirs(output_path)
    
summary_path = os.path.join(output_path, "summaries")
paths['summary_path'] = summary_path

if not os.path.exists(summary_path): os.makedirs(summary_path)
model_path = os.path.join(output_path, "checkpoints/")
if not os.path.exists(model_path): os.makedirs(model_path)
ckpt_prefix = os.path.join(model_path, "model")
paths['model_path'] = ckpt_prefix

result_path = os.path.join(output_path, "results")
paths['result_path'] = result_path

if not os.path.exists(result_path): os.makedirs(result_path)
log_path = os.path.join(result_path, "log.txt")
paths['log_path'] = log_path

get_logger(log_path).info(str(args))

print(paths['result_path'])

vocab_size: 4271


Namespace(CRF=True, batch_size=64, clip=5.0, demo_model='1559725764', dropout=0.5, embedding_dim=300, epoch=40, hidden_dim=300, lr=0.001, mode='demo', optimizer='Adam', pretrain_embedding='random', shuffle=True, test_data='data_path', train_data='data_path', update_embedding=True)


./data_path_save/1559725764/results


In [8]:
#USE MODEL
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
#build_graph用一次就好了~
model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
model.build_graph()

./data_path_save/1559725764/checkpoints/model-633400
Instructions for updating:
Colocations handled automatically by placer.


From /Users/rikeion/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


From /Users/rikeion/Desktop/Anti_Money_Laundry/zh-NER-TF-master/model.py:60: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


...

From /Users/rikeion/Desktop/Anti_Money_Laundry/zh-NER-TF-master/model.py:71: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


From /Users/rikeion/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/rnn.py:443: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Instructions for updating:
Use tf.cast instead.


From /Users/rikeion/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/rnn.py:626: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
/Users/rikeion/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [9]:
def ner_process_document(article):
    #函數用來呼叫NER，讓NER能被多次使用
    data = list()
    saver = tf.train.Saver()
    with tf.Session(config=config) as sess:
        saver.restore(sess, ckpt_file)
        demo_sent = article.strip()
        demo_data = [(demo_sent, ['O'] * len(demo_sent))]
        tag = model.demo_one(sess, demo_data)
        PER, LOC, ORG = get_entity(tag, demo_sent)
        data.append({
            'article' : article,
            'per' : PER,
            'loc' : LOC,
            'org' : ORG
        })
    return data

In [10]:
def read_given_name():
    #導入四百家姓
    with open("../given_name_list", "r") as r1:
        given_name_list = []
        for i in r1:
            given_name_list.append(i.strip("\n"))
    return given_name_list

In [11]:
def scan_name(per_list, given_name_list):
    #函數用四百家姓及字數等規則來過濾NER判斷失敗的地方
    new_per_list = []
    for i in per_list:
        if len(i) < 2 or len(i) > 4:
            print(1)
            continue
        elif i[0] not in given_name_list:
            print(2)
            continue
        elif not all('\u4e00' <= char <= '\u9fff' for char in i):
            print(3)
            continue
        else: 
            new_per_list.append(i)
            
    return list(set(new_per_list))

In [12]:
def tradition2simple(sentence):
    #函數將繁體中文轉成簡體中文
    cc = OpenCC('t2s')
    sentence = cc.convert(sentence)
    return sentence

In [13]:
def simple2tradition(sentence):
    #函數將簡體中文轉成繁體中文
    cc = OpenCC('s2t')
    sentence = cc.convert(sentence)
    return sentence

In [14]:
def ner_demo(sentence):
    #組合上面函數，使用NER並加入判斷是否是名字，輸出名字的list
    try:
        #sentence = tradition2simple(sentence)
        data = ner_process_document(sentence)
        per_list = data[0]['per']
        given_name_list = read_given_name()
        new_per_list = scan_name(per_list, given_name_list)
        print(per_list)
        #new_per_list = #list(map(simple2tradition,new_per_list))
    except:
        return "ERROR"
    return new_per_list


# 找出同時包含名字及關鍵字之句子

In [15]:
def get_sentence(article,keyword_list,name_list):
    #函數用來找出同時包含名字及洗錢關鍵字的句子
    sentence_contain_keyword = []
    for i in range(len(article)):
        for j in range(len(keyword_list)):
            if article[i].find(keyword_list[j]) != -1:
                sentence_contain_keyword.append(article[i])            
    sentence_contain_keyword = set(sentence_contain_keyword)
    
    sentence_contain_name = []
    for i in range(len(article)):
        for j in range(len(name_list)):
            if article[i].find(name_list[j]) != -1:
                sentence_contain_name.append(article[i])            
    sentence_contain_name = set(sentence_contain_name)
    
    sentence_wanted = set(sentence_contain_keyword) & set(sentence_contain_name)
    sentence_wanted = list(sentence_wanted)
    return sentence_wanted

# 將名字及關鍵字寫入自訂辭典

In [16]:
def write_word(word,freq=10):
    #將名字以及關鍵字寫入jieba辭典裡，以提高切詞正確率
    with open('/Users/rikeion/Desktop/Anti_Money_Laundry/user_dict.txt','r') as f:
        readword = f.read()
        if word not in readword:
            with open('/Users/rikeion/Desktop/Anti_Money_Laundry/user_dict.txt', 'a') as file:
                file.write("\n"+word+" "+str(freq))

# 匯入自訂辭典

In [17]:
jieba.load_userdict("/Users/rikeion/Desktop/Anti_Money_Laundry/user_dict.txt")

Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/4s/mykg15b9135fc0jvn3h80fl00000gn/T/jieba.cache
Loading model from cache /var/folders/4s/mykg15b9135fc0jvn3h80fl00000gn/T/jieba.cache
Loading model cost 1.547 seconds.
Loading model cost 1.547 seconds.
Prefix dict has been built succesfully.
Prefix dict has been built succesfully.


# 用jieba切字並將句子製成樹狀

In [18]:
def make_tree(sentence,keyword_list):
    #先判斷句子中是否含有關鍵字，若無則回傳None;接著，將句子做成樹狀結構，以便解析句構
    token = jieba.cut(sentence, cut_all = False)
    flag = 0
    token_list = list(token)
    for i in keyword_list: 
        if simple2tradition(i) in token_list:
            flag = 1
    if flag!=1:
        return None
    string = '-'.join(token_list)
    string = tradition2simple(string)    #轉簡體字
    jieba_string = list(parser.parse_text(string))
    string = str(jieba_string[0]).replace('\n','')
    string = string.replace('(PU -)','')
    string = string.split(" ")
    a = str()
    for i in string:
        if i !=" " and i!='':
            a += i
            a += " "
    a = a.replace('(PU -)','')
    tree = Tree.fromstring(a)
    return tree

# 句構解析

In [19]:
def count_distance(string_tree,from_leaf,to_leaf):
    #計算兩個節點之距離
    try:
        tree_pos = string_tree.treepositions('leaves')
        find_node = ()
        for i in tree_pos:
            if string_tree[i]==from_leaf:
                from_leaf_idx = i
            elif string_tree[i]==to_leaf:
                to_leaf_idx = i
        length = len(from_leaf_idx) if len(from_leaf_idx)<len(to_leaf_idx) else len(to_leaf_idx)
        for i in range(length):
            if from_leaf_idx[i]==to_leaf_idx[i]:
                find_node += (from_leaf_idx[i],)
            else:
                break
        length_from_leaf = len(from_leaf_idx) - len(find_node)
        length_to_leaf = len(to_leaf_idx) - len(find_node)
        return length_from_leaf+length_to_leaf
    except:
        return 'Error'

In [20]:
def find_idx(string_tree,name):
    #找尋特定節點之位置
    tree_pos = string_tree.treepositions('leaves')
    for i in tree_pos:
        if string_tree[i]==name:
            name_idx = i
    return name_idx

In [21]:
def is_same_label(tree, name_1,name_2):
    #判斷兩個字在共同節點之下一層節點之label皆為NP或是皆為IP
    name1 = find_idx(tree,name_1)
    name2 = find_idx(tree,name_2)
    find_node_1 = []
    find_node_2 = []
    for i in range(len(name1)):
        if name1[i] == name2[i]:
            find_node_1 += [name1[i]]
            find_node_2 += [name2[i]]
        else:
            find_node_1 += [name1[i]]
            find_node_2 += [name2[i]]
            break
    if (tree[find_node_1].label() == 'NP' and tree[find_node_2].label() == 'NP') or (tree[find_node_1].label() == 'IP' and tree[find_node_2].label() == 'IP'):
        return True
    else:
        return False

In [22]:
def is_leaf(tree,word):
    #判斷特定字是否為樹之子葉
    tree_pos = tree.treepositions('leaves')
    for i in tree_pos:
        if tree[i]==word:
            return True
    return False

In [23]:
def find_bad(tree, name_list, keyword_list):
    #當名字多於一人時，判斷哪些名字為類似之名字，i.e. 顧立雄控訴吳志揚以及趨同道在洗錢，則吳志揚以及趨同道為同一類，而顧立雄為另一類
    name_bad = {}
    distance_list = []
    closest_name_list = []
    if tree==None:
        return None
    for i in keyword_list:
        if is_leaf(tree,i):
            keyword = i
    if len(name_list) > 1:
        for idx, i in enumerate(name_list):
            name_bad[i] = False
            distance_list.append(count_distance(tree,i,keyword))
#                 if distance_list[idx]=='Error':
#                     name_bad[i] = None
#             if all(i=='Error' for i in distance_list):
#                 return closest_name_list
        minidx = np.argmin(distance_list)
#             print("minidx= ",minidx)
        name_bad[name_list[minidx]] = True
        for i in range(len(name_list)):
            if i != minidx:
                dist1 = count_distance(tree,name_list[minidx],keyword)
                dist2 = count_distance(tree,name_list[i],keyword)
                if dist2=='Error':
                    continue
                elif dist1 != dist2:
                    if is_same_label(tree,name_list[minidx],name_list[i]):
                        name_bad[name_list[i]] = True
                elif dist1 == dist2: #and is_same_label(tree,name_list[minidx],name_list[i]): #距離一樣且label一樣
                    name_bad[name_list[i]] = True
        for i in name_bad.items():
            if i[1]==True:
                closest_name_list.append(i[0])
    elif len(name_list)==0:
        return []
    else:
        closest_name_list.append(name_list[0])
    return closest_name_list

In [24]:
def judge_active_verb(verb):
    #函數判斷是否具有主動的洗錢動詞
    r1 = open('../money_laundring_verb.txt', 'r')
    verb_list = []
    for i in r1:
        verb_list.append(tradition2simple(i.strip()))
    proved_verb = "n"
    if verb in verb_list:
        proved_verb = "y"
    r1.close()
    return proved_verb

In [25]:
def judge_passive_verb(verb):
    #函數判斷是否具有被動的洗錢動詞
    r1 = open('../money_laundring_passive_verb.txt', 'r')
    verb_list = []
    for i in r1:
        verb_list.append(tradition2simple(i.strip()))
    proved_verb = "n"
    if verb in verb_list:
        proved_verb = "y"
    r1.close()
    return proved_verb

In [26]:
def judge_sentence(tree,show_name, closest_criminal_list, keyword_list):
    #函數用來判斷句構是否是：名字 +廢字+ 動詞（有洗錢嫌疑的動詞） + 關鍵字/包含關鍵字的子句或廢字
    if len(closest_criminal_list) != 0:
        closest_criminal = closest_criminal_list[-1]
    try:
        leaves_upper_node_indexs = []
        for i in tree.treepositions('leaves'):#treepositions('leaves')的list裡面是tuple
            index = []
            for j in range(len(i)-1):
                index.append(i[j])
            index = tuple(index)
            leaves_upper_node_indexs.append(index)
        words_and_tags = []
        for i in leaves_upper_node_indexs:
            words_and_tags.append(str(tree[i]).strip("(").strip(")").split(" "))
        #print(words_and_tags)#每一個項目長這樣： ['NR', '桃園']
        #找出離名字最近的動詞(這裡可能之後會有問題，但在句中離名字最近會比用樹的距離來判斷好)

        #檢查是否名字在受詞
        passive = False
        closest_criminal_index = 0
        keyword_index = 0
        how_close = 10000
        for i in words_and_tags:
            if i[1] == closest_criminal:
                closest_criminal_index = words_and_tags.index(i)
                break
        for i in words_and_tags:
            for j in keyword_list:
                if i[1] == j and abs(keyword_index - closest_criminal_index) < how_close :
                    keyword_index = words_and_tags.index(i)
                    how_close = abs(keyword_index - closest_criminal_index)
        if keyword_index < closest_criminal_index:
            passive = True


        #若名字在敘事句的主詞位置
        if passive == False:
            name_flag = 0 #假設前面沒有出現過那個名字（一般一個敘事句會提到兩次同個人名字的可能比較小）
            bad_sentence = False
            v = ''
            pass_word = 0
            pass_word_list = []
            for i in words_and_tags:
                if i[1] == closest_criminal:
                    name_flag += 1
                if name_flag == 1:#目的是判斷現在句子位置在要找的人名之後
                    if i[0] == 'VV':
                        v = i[1]
                        if judge_active_verb(v) == 'y':
                            bad_sentence = True
                        break
                    else:
                        pass_word_list.append(i[1])
            pass_word = len(pass_word_list)

        #若名字在敘事句的受詞位置（往回查找）
        if passive == True:
            bad_sentence = False
            v = ''
            pass_word = 0
            pass_word_list = []
            for i in range(closest_criminal_index):
                if words_and_tags[-(len(words_and_tags)-closest_criminal_index)+(-i)][0] == 'VV':
                    v = words_and_tags[-(len(words_and_tags)-closest_criminal_index)+(-i)][1]
                    if judge_passive_verb(v) == "y":
                        bad_sentence = True
                    break
                else:
                    pass_word_list.append(words_and_tags[-(len(words_and_tags)-closest_criminal_index)+(-i)][1])
            pass_word = len(pass_word_list)
        name_and_status = dict()
        for j in show_name:
                name_and_status[j] = False
        for i in closest_criminal_list:
            name_and_status[i] = bad_sentence
    except:
        name_and_status = "ERROR"
    return name_and_status#[bad_sentence, v, pass_word, pass_word_list]

# 計算正確率

In [27]:
def evaluate_accuracy_s(show_name, name_dict, ans_dict):
    wrong_s = []
    accuracy=0
    for j in range(len(name_dict)):
        flag=0
        for i in show_name[j]:
            try:
                if name_dict[j][i]!=ans_dict[j][i]:
                    flag=0
                    wrong_s.append(j)
                    break
                flag=1
            except:
                print("i am except")
                continue
    
        accuracy+=flag    
            
    return accuracy/float(len(name_dict)), wrong_s

In [28]:
def evaluate_accuracy_w(show_name, name_dict, ans_dict):
    accuracy=0
    total_name = 0
    for j in range(len(name_dict)):
        flag=0
        total_name+=len(show_name[j])
        for i in show_name[j]:
            try:
                if name_dict[j][i]==ans_dict[j][i]:
                    flag+=1
            except:
                continue
        accuracy+=flag
    return accuracy/total_name  

# 資料測試

In [29]:
def organize_data():
    #整理測試資料
    text_data = []
    with open('../洗錢測試50句資料csv3.csv','r') as r3:
        for i in r3:
            text_data.append(i.strip('"').strip())
    text_data.remove(text_data[0])
    text_data1 = []
    for i in text_data:
        text_data1.append(i.split(","))

    for i in range(len(text_data1)):
        text_data1[i][1] = text_data1[i][1].split("，")
        text_data1[i][0] = text_data1[i][0].split("，")
    sentence_wanted = [text_data1[i][2] for i in range(len(text_data1))]
    show_name = []
    for i in range(len(text_data1)):
        show_name.append(text_data1[i][1])
    ans_dict = []
    for i in range(len(text_data1)):
        answer_dict = dict()
        for j in text_data1[i][0]:
            if j in answer_dict:
                no_use = 0
            else:
                answer_dict[j] = True

        for k in text_data1[i][1]:
            if k in answer_dict:
                no_use = 0
            else:
                answer_dict[k] = False
        #del answer_dict["無"] 
        if "無" in answer_dict:
            del answer_dict["無"]
        ans_dict.append(answer_dict)
    name_in_sentence = []
    for i in sentence_wanted:
        name = ner_demo(i)
        name_in_sentence.append(name)
        
    return sentence_wanted, name_in_sentence, ans_dict

# Main

In [97]:
def demo():
#     try:
    keyword_list = ["賄賂","貪污", "洗錢", "詐欺", "境外轉帳", "詐騙"]
    sentence_wanted, show_name, ans_dict = organize_data()
    name_dict = []
    show_name_s_list = list()
    #加入結巴詞庫
    for i in show_name:
        for j in i:
            write_word(j)

    for i in keyword_list:
        write_word(i)
    keyword_list = list(map(tradition2simple, keyword_list))   #把keyword轉簡體
    #讀取結巴
    jieba.load_userdict("/Users/rikeion/Desktop/Anti_Money_Laundry/user_dict.txt")
    for i,sentence in enumerate(sentence_wanted):
        show_name_s = list(map(tradition2simple, show_name[i]))
        show_name_s_list.append(show_name_s)
        parser = CoreNLPParser('http://localhost:9001')
        name = {}
        tree = make_tree(sentence,keyword_list)
        if tree!=None:
            closest_name_list = find_bad(tree, show_name_s, keyword_list)
        else:
            closest_name_list = None
        if closest_name_list == None:
            name = {j:False for j in show_name_s}
        else:
            name = judge_sentence(tree, show_name_s, closest_name_list, keyword_list)
        name_dict.append(name)
    for k in ans_dict:
        for i in k.keys():
            k[tradition2simple(i)] = k.pop(i)
    for k in ans_dict:
        for i in k.keys():
            k[tradition2simple(i)] = k.pop(i)
    accuracy1,wrong_s = evaluate_accuracy_s(show_name_s_list,name_dict,ans_dict)
    print("Accuracy by sentence = ",accuracy1)   
    accuracy2 = evaluate_accuracy_w(show_name_s_list,name_dict,ans_dict)
    print("Accuracy by name = ",accuracy2) 
    print(np.take(sentence_wanted,wrong_s))
    return name_dict
#     except:
#         print("Error :(((")

In [105]:
#計入NER後的成果:65%
name_dict = demo()

INFO:tensorflow:Restoring parameters from ./data_path_save/1559725764/checkpoints/model-633400


Restoring parameters from ./data_path_save/1559725764/checkpoints/model-633400


1
['吳志揚', '鄒德道', '鄒']
INFO:tensorflow:Restoring parameters from ./data_path_save/1559725764/checkpoints/model-633400


Restoring parameters from ./data_path_save/1559725764/checkpoints/model-633400


['林志吉']
INFO:tensorflow:Restoring parameters from ./data_path_save/1559725764/checkpoints/model-633400


...

Restoring parameters from ./data_path_save/1559725764/checkpoints/model-633400


['賴英門']


Resetting dropped connection: localhost
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 74418
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 61632
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 209483
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 23740
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 134991
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2

http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 35953
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 75277
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 70948
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 53392
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 21605
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1

i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
i am except
Accuracy by sentence =  0.6422764227642277
Accuracy by name =  0.6530612244897959
["桃園地檢署偵辦前桃園縣長吳志揚隨行秘書鄒德道等人涉嫌收受楊梅地區的地主100萬元現金及價值200萬元的BMW案，並居中介紹官員協助土地變更案，今偵查終結，依貪污治罪條例起訴將鄒等人起訴'"
 '\\xa0檢方認為，吳宣在2018年6月間加入孫武生販毒集團，協助運送裝有毒品的菸盒到買家指定地點，並替孫男拿取買家投放孫男刺青工作室信箱裝有金錢的信封，涉嫌洗錢，還替孫武生、班特到萬華區刀品店購買殺害顏伯萊的兩把開山刀、鏈鋸等，案發時更在命案現場永和河濱公園對岸把風，後因看不清楚，孫男才讓吳宣先行離開'
 '美國司法部在台灣時間今晨4時許高調舉行記者會，公佈起訴華為、華為子公司及華為副董事長孟晚舟共23項罪名，包括銀行詐欺、密謀洗錢、竊取美國公司機密等，並宣布會向加拿大要求引渡孟晚舟'
 '\\xa0已故台北市議員李新前女友郭新政成立「誰摔死了李新」臉書專頁，至今已推出9支影片，並找來資深主播盛竹如拍片，影射前立委羅淑蕾涉入郭新政的珠寶詐騙案害死李新，羅對郭與盛提告，但是事情並沒有告一段落，郭新政指控羅淑蕾找人電話恐嚇盛竹如，劇情堪比連續劇'
 '由洪俊凱在網路搜尋泰國電話號碼，每次設定500至1000組號碼後，以群呼系統傳送語音訊息給泰國民眾，佯稱電話號碼將被封鎖，當民眾詢問時，即轉接至詐騙成員手機，誆騙對方涉及洗錢、販毒，銀行存款要匯出列管等'
 '檢方偵辦中山分局警員涉嫌集體收賄包庇酒店，查出日式酒店「曉」及其前身「立邦酒店」每月提供4萬元現金給轄區警方，換取包庇、通風報信，13年間共收賄780萬元，北檢去年7月依違反《貪污治罪條例》的行賄罪、妨害風化罪起

In [109]:
#不計入NER正確率之前的成果：72%
name_dict1 = demo()

Resetting dropped connection: localhost
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 61632
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 209483
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 23740
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 133644
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 77380
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2

http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 75277
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 70948
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 53392
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 21605
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1.1" 200 40941
http://localhost:9001 "POST /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%7D HTTP/1

Accuracy by sentence =  0.6666666666666666
Accuracy by name =  0.7208121827411168
["桃園地檢署偵辦前桃園縣長吳志揚隨行秘書鄒德道等人涉嫌收受楊梅地區的地主100萬元現金及價值200萬元的BMW案，並居中介紹官員協助土地變更案，今偵查終結，依貪污治罪條例起訴將鄒等人起訴'"
 '\\xa0檢方認為，吳宣在2018年6月間加入孫武生販毒集團，協助運送裝有毒品的菸盒到買家指定地點，並替孫男拿取買家投放孫男刺青工作室信箱裝有金錢的信封，涉嫌洗錢，還替孫武生、班特到萬華區刀品店購買殺害顏伯萊的兩把開山刀、鏈鋸等，案發時更在命案現場永和河濱公園對岸把風，後因看不清楚，孫男才讓吳宣先行離開'
 '美國司法部在台灣時間今晨4時許高調舉行記者會，公佈起訴華為、華為子公司及華為副董事長孟晚舟共23項罪名，包括銀行詐欺、密謀洗錢、竊取美國公司機密等，並宣布會向加拿大要求引渡孟晚舟'
 '\\xa0已故台北市議員李新前女友郭新政成立「誰摔死了李新」臉書專頁，至今已推出9支影片，並找來資深主播盛竹如拍片，影射前立委羅淑蕾涉入郭新政的珠寶詐騙案害死李新，羅對郭與盛提告，但是事情並沒有告一段落，郭新政指控羅淑蕾找人電話恐嚇盛竹如，劇情堪比連續劇'
 '由洪俊凱在網路搜尋泰國電話號碼，每次設定500至1000組號碼後，以群呼系統傳送語音訊息給泰國民眾，佯稱電話號碼將被封鎖，當民眾詢問時，即轉接至詐騙成員手機，誆騙對方涉及洗錢、販毒，銀行存款要匯出列管等'
 '檢方偵辦中山分局警員涉嫌集體收賄包庇酒店，查出日式酒店「曉」及其前身「立邦酒店」每月提供4萬元現金給轄區警方，換取包庇、通風報信，13年間共收賄780萬元，北檢去年7月依違反《貪污治罪條例》的行賄罪、妨害風化罪起訴酒店業者巫蕙玲等3人，並以收賄罪起訴中山一派出所所長林崇成、警員陳宏洲、紀炳場、郜振傑等10警員，其中3警認罪，但有7警堅不認罪，因此請求法院重判7警，另林崇成被查出將賄款存入妻子李青芬帳戶，檢方另依違反《洗錢防治條例》一併起訴'
 '而在去年台北市議員李新跳樓自殺周年忌日，郭新政到台北地檢署，指控羅淑蕾與珠寶商嚴嘉慧偽造當票，害她及李新被外界抹黑指責是詐欺犯、洗錢犯'
 '何志平前天身穿深色囚衣出庭聆聽判決，比昔日消瘦不少，法官指出，貪污是嚴重罪行